# Load, preprocess, and save train and test data
# John Brandt
# April 1, 2020

- Fuse Sentinel 1/2 data
- Reconstruct 2D-array from CEO output CSV by plot
- Match sentinel data to CEO labels
- Stack data_x, data_y, length
- Save numpy arrays for data_x, data_y, length

The notebook additionally contains some development code for:
- Parameter selection in whittaker smoothing
- Graphing plot locations on map

# Package imports and source code

In [1]:
#TODO Remove imports that aren't needed to save RAM
from tqdm import tqdm_notebook, tnrange

import pandas as pd
import numpy as np
from random import shuffle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random
import itertools

os.environ['KMP_DUPLICATE_LIB_OK']='True'

#!source ~/.bash_profile
#!export PATH="/usr/lib/google-cloud-sdk/bin:$PATH"

%run ../src/utils/utils.py
%run ../src/utils/slope.py

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
def reconstruct_images(plot_id):
    '''Takes a plot ID and subsets the input pd.DataFrame to that plot ID
       returns a (14, 14) array-like list with binary labels
       
        Parameters:
          batch_ids (list):
          batch_size (int):
          
         Returns:
          x_batch (arr):
          y_batch (arr):
    '''
    subs = df[df['PLOT_ID'] == plot_id]
    rows = []
    lats = reversed(sorted(subs['LAT'].unique()))
    for i, val in enumerate(lats):
        subs_lat = subs[subs['LAT'] == val]
        subs_lat = subs_lat.sort_values('LON', axis = 0)
        rows.append(list(subs_lat['TREE']))
    return rows

In [54]:
source = 'test'
sentinel_1 = True
s2_path = "../data/{}-s2/".format(source)
s1_path = "../data/{}-s1/".format(source)
csv_path = "../data/{}-csv/".format(source)
output_path = "../data/{}-processed/".format(source)

#s2_path = "../data/drylands/s2/"
#s1_path = "../data/drylands/s1/"
#csv_path = "../data/drylands/csv/"

#s2_path = "../data/project-val/sentinel-2/"
#s1_path = "../data/project-val/sentinel-1/"
#csv_path = "../data/project-val/"

In [55]:
# Regional models
east_africa = ['ethiopia', 'africa-east', 'hyperarid', 'kenya',
               'malawi', 'rwanda', 'subplot', 'sudan', 'africaeast',] # 1442, 236

west_africa = ['africa-west', 'ghana', 'niger', 'koure', 
               'hyperarid', 'africawest', 'cameroon'] # 1685, 171

asia = ['australia', 'asia', 'india', 'global'] # 471, 346

americas = ['brazil', 'elsalvador', 'honduras', 
            'lac', 'mexico', 'sa-', 'america'] # 976, 218

africa = east_africa + west_africa

malawi = ['malawi', 'africaeast']
nigercameroon = ['niger', 'koure', 'cameroon']

globe = [x for x in os.listdir(csv_path)]

region = globe

In [56]:
# For either train or test data, loop through each plot and determine whether there is
# labelled Y data for it -- returning one dataframe for the entire data set

dfs = []
for i in os.listdir(csv_path):
    if ".csv" in i:
        if any([x in i for x in region]):

            print(i)
            df = pd.read_csv(csv_path + i).drop('IMAGERY_TITLE', axis = 1)
            df['country'] = i.split(".")[0]
            dfs.append(df)

for i in range(len(dfs)):
    if "PL_PLOTID" not in dfs[i].columns:
        dfs[i]['PL_PLOTID'] = 0
            #dfs[i] = dfs[i].drop("PL_PLOTID", axis = 1)
    if 'STACKINGPROFILEDG' in dfs[i].columns:
        dfs[i] = dfs[i].drop('STACKINGPROFILEDG', axis = 1)
    if 'IMAGERYYEARDG' in dfs[i].columns:
        dfs[i] = dfs[i].drop('IMAGERYYEARDG', axis = 1)

df = pd.concat(dfs, ignore_index = True)
df = df.dropna(axis = 0)

existing = [int(x[:-4]) for x in os.listdir(s2_path) if ".DS" not in x]

df = df[df['PLOT_ID'].isin(existing)]
plot_ids = sorted(df['PLOT_ID'].unique())

africaoceana-test.csv
india-test.csv
lac-north-test.csv
cameroon-test.csv
europe-test.csv
australia-test.csv
eurasia-test.csv
americas-test-random.csv
ghana-test.csv
global-test.csv
kenya-test.csv
lac-south-test.csv
ethiopia-test.csv
ghana-test-large.csv
africaeast-test.csv
africawest-test.csv


/Users/john.brandt/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [57]:
# Initiate empty lists to store the X and Y data in
data_x, data_y, lengths = [], [], []
    
    
countries = {}
count = 0
to_remove = []
plot_ids_loaded = []
pl_plot_ids_loaded = []
# Iterate over each plot
for i in tnrange(len(plot_ids)):
    skip = True if sentinel_1 else False
    # Load the sentinel imagery
    if (str(plot_ids[i]) + ".npy")in os.listdir(s2_path):
        if plot_ids[i] not in [136077593, 136776935, 136776898]:
            country = df[df['PLOT_ID'] == plot_ids[i]]['country'].unique()
            if str(country[0]) not in countries.keys():
                countries[str(country[0])] = [count, count]
            countries[str(country[0])][1] = count
            x = np.load(s2_path + str(plot_ids[i]) + ".npy")
            if sentinel_1 and os.path.isfile(s1_path + str(plot_ids[i]) + ".npy"):
                skip = False
                s1 = np.load(s1_path + str(plot_ids[i]) + ".npy")
                print(np.max(s1))
                x = np.concatenate([x, s1], axis = -1)
            count += 1
        y = reconstruct_images(plot_ids[i])
        if not skip:
            plot_ids_loaded.append(plot_ids[i])
            lengths.append(x.shape[0])
            data_x.append(x)
            data_y.append(y)
print("Finished data loading")

data_x = np.stack(data_x)
data_y = np.stack(data_y)
lengths = np.stack(lengths)

0.7358052948805982
0.7730983443961242
1.0
1.0
1.0
0.8966964217593653
0.895002670328832
0.7951018539711604
0.8574959945067521
0.6493018997482262
0.4242008087281605
0.7347066453040361
0.46514839398794533
0.8295490959029527
0.5237964446478981
0.9274509803921569
1.0
0.8793011367971313
0.9571068894483864
1.0
1.0
1.0
0.7472953383688106
0.867002365148394
1.0
1.0
1.0
1.0
0.598992904554818
1.0
1.0
0.8634927901121539
1.0
1.0
1.0
0.7837033646143282
0.7109941252765698
1.0
1.0
0.7622034027618829
0.9374074921797513
0.7544060425726711
1.0
0.5859998474097811
0.8502937361715114
0.6630045014114595
1.0
1.0
0.5226062409399558
0.6106965743495841
1.0
0.9031052109559777
0.6836041809719997
1.0
0.7940947585259784
1.0
1.0
0.7620050354772259
0.4068055237659266
1.0
0.9781948577096208
0.6686961165789272
0.8771953917753872
0.7647058823529411
1.0
0.893797207598993
0.8955062180514229
0.5023498893720912
1.0
0.7479972533760586
0.9539940489814603
0.49129472800793467
0.858503089951934
0.8892042420080872
0.685648889906157

In [58]:
print(data_x.shape)

(260, 24, 16, 16, 17)


In [59]:
countries

{'india-test': [0, 50],
 'lac-north-test': [51, 91],
 'cameroon-test': [92, 130],
 'africaoceana-test': [131, 196],
 'europe-test': [197, 261]}

In [60]:
if source == 'train':
    np.save("../tile_data/processed/data_x_l2a_processed.npy", data_x)
    np.save("../tile_data/processed/data_y_l2a_processed.npy", np.array(data_y))
    np.save("../tile_data/processed/length_l2a_processed.npy", np.array(lengths))
if source == 'test' or source == "project":
    print("Writing test data")
    np.save("../tile_data/processed/test_x_l2a_processed.npy", data_x)
    np.save("../tile_data/processed/test_y_l2a_processed.npy", data_y)
    np.save("../tile_data/processed/test_length_l2a_processed.npy", lengths)

Writing test data


# Development code

## Parameter evaluation for lambda in whittaker smoother

In [107]:
def mask_segments(arr, percent):
    '''Docstring
    
         Parameters:
          arr (arr):
          percent (float):
          
         Returns:
          masked (arr):
    '''
    masked = np.copy(arr)
    for sample in range(masked.shape[0]):
        start = random.sample(range(72 - int(72*(percent))), 1)[0]
        length = int(72*(percent))
        
        masked[sample, start:start+length] = np.tile(
            (masked[sample, start] + masked[sample, start+length])/2,
            (1, length, 1, 1, 1))
        
    return masked

def apply_smoothing(arr, lmb):
    '''Docstring
    
         Parameters:
          arr (arr):
          lmb (float):
          
         Returns:
          smoothed (arr):
    '''
    smoothed = np.copy(arr)
    for sample in tnrange(smoothed.shape[0]):
        for row in range(0, 16):
            for column in range(0, 16):
                for band in [x for x in range(0, 15) if x != 10]:
                    sm = smooth(smoothed[sample, :, row, column, band], lmb, d = 2)
                    smoothed[sample, :, row, column, band] = sm
    return smoothed

#for lmb in [1, 20, 50, 100, 500, 750]:
#    for percent in [20, 30, 40, 50]:
#        masked = mask_segments(data_x, percent/100)
#        smoothed = apply_smoothing(masked, lmb)
#        mse = np.mean(abs(smoothed - data_x))
#        #mse = calculate_mse(smoothed, data_x)
#        print("{}% masked data, {} lambda: {} error".format(percent, lmb, mse))

## Extraction of lat and longs for training / testing data

In [ ]:
lats = []
longs = []
plot_ids = []
df = pd.read_csv("../data/science-2017-test.csv")
df = df.sample(frac=1, random_state = 5)
df['PLOT_ID'] = range(0, len(df), 1)
df['group'] = df['tree_cover'] * 100
print(np.unique(df['group']))
df['group'] = df['group'].astype(int)
print(np.unique(df['group']))
for i in range(0, len(df), 100):
    lat = df['location_y'][i]
    lon = df['location_x'][i]
    lats.append(lat)
    longs.append(lon)
    plot_ids.append(i)

In [10]:
lats = []
longs = []
for i in plot_ids_loaded:
    subs = df[df['PLOT_ID'] == i]
    lat = float(subs['LAT'].head(1))
    lon = float(subs['LON'].head(1))
   # print(subs['LAT'])
    #if lon > region[0] and lon < region[1]:
        #if lat > region[2] and lat < region[3]:
    lats.append(lat)
    longs.append(lon)
    
lats = np.array(lats)
longs = np.array(longs)
    
BBox = ((longs.min(),   longs.max(),      
         lats.min(), lats.max()))

print(BBox)
print(lats.shape)

(-123.7612999908743, 156.998167669016, -54.904626104679984, 67.53857583112786)
(1007,)


In [11]:
#np.save("../data/metrics/plotids.npy", plot_ids_loaded)
testing_latlongs = pd.DataFrame({'lats':lats,'longs':longs, 'id':plot_ids_loaded})
testing_latlongs.to_csv("../data/drylands/latlong.csv", index = False)